In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
# img_width, img_height = 224, 224
img_width, img_height = 500, 500

pretrained_model = "inceptionv3_500_small"


top_model_weights_path = '../model/bottleneck_fc_model_%s.h5'%pretrained_model
train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
test_data_dir = '../data/Pandora18K_small_train_val_test_split/test'
nb_train_samples = 1462
nb_validation_samples = 167
nb_test_samples = 171
# train_data_dir = '../data/Pandora18K_train_val_test_split/train'
# validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
# test_data_dir = '../data/Pandora18K_train_val_test_split/test'
# nb_train_samples = 14313
# nb_validation_samples = 1772
# nb_test_samples = 1791

epochs = 50
batch_size = 64



Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(rescale=1. / 255)
# build the VGG16 network
# model = applications.VGG16(include_top=False, weights='imagenet')
model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet')

In [3]:
from keras.utils.np_utils import to_categorical

train_data = np.load(open('../model/bottleneck_features_train_%s.npy'%pretrained_model))
#full
#train_labels = np.array([0]*684+[1]*598+[2]*655+[3]*657+[4]*808+[5]*675+[6]*715+[7]*946+[8]*995+[9]*1021+[10]*803+[11]*816+[12]*566+[13]*959+[14]*842+[15]*831+[16]*849+[17]*893)

#small
train_labels = np.array([0]*78+[1]*79+[2]*77+[3]*85+[4]*78+[5]*87+[6]*80+[7]*81+[8]*81+[9]*80+[10]*85+[11]*83+[12]*74+[13]*87+[14]*83+[15]*83+[16]*78+[17]*83)

train_labels = to_categorical(train_labels, num_classes=18)

validation_data = np.load(open('../model/bottleneck_features_validation_%s.npy'%pretrained_model))
#full
#validation_labels = np.array([0]*72+[1]*73+[2]*72+[3]*93+[4]*78+[5]*74+[6]*85+[7]*124+[8]*131+[9]*118+[10]*109+[11]*105+[12]*80+[13]*130+[14]*108+[15]*89+[16]*111+[17]*120)

#small
validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
validation_labels = to_categorical(validation_labels, num_classes=18)

test_data = np.load(open('../model/bottleneck_features_test_%s.npy'%pretrained_model))
#full
#test_labels = np.array([0]*91+[1]*60+[2]*75+[3]*82+[4]*104+[5]*83+[6]*95+[7]*121+[8]*131+[9]*123+[10]*103+[11]*117+[12]*65+[13]*123+[14]*121+[15]*112+[16]*89+[17]*96)

#small
test_labels = np.array([0]*14+[1]*11+[2]*12+[3]*10+[4]*11+[5]*7+[6]*12+[7]*11+[8]*10+[9]*8+[10]*8+[11]*7+[12]*12+[13]*8+[14]*6+[15]*6+[16]*10+[17]*8)
test_labels = to_categorical(test_labels, num_classes=18)

In [8]:
import keras
import os
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dropout(0.6))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(18, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='categorical_crossentropy', metrics=['accuracy'])


weight_dir = "../model/%s"%pretrained_model
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
weight_path = os.path.join(weight_dir, "best_weights%s.h5"%pretrained_model)
checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path), verbose=1,monitor='val_acc', save_best_only=True)
model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels),callbacks=[checkpointer])


# model.save_weights(top_model_weights_path,overwrite=True)


Train on 1462 samples, validate on 167 samples
Epoch 1/50
1462/1462 [==============================] - 10s - loss: 3.5089 - acc: 0.0896 - val_loss: 2.7344 - val_acc: 0.0898
Epoch 2/50
1462/1462 [==============================] - 18s - loss: 2.6153 - acc: 0.1546 - val_loss: 2.6487 - val_acc: 0.1737
Epoch 3/50
1462/1462 [==============================] - 17s - loss: 2.4376 - acc: 0.2038 - val_loss: 2.4732 - val_acc: 0.2036
Epoch 4/50
1462/1462 [==============================] - 6s - loss: 2.4029 - acc: 0.2038 - val_loss: 2.5204 - val_acc: 0.1856
Epoch 5/50
1462/1462 [==============================] - 18s - loss: 2.2498 - acc: 0.2387 - val_loss: 2.5029 - val_acc: 0.2275
Epoch 6/50
1462/1462 [==============================] - 6s - loss: 2.2031 - acc: 0.2606 - val_loss: 2.4008 - val_acc: 0.2216
Epoch 7/50
1462/1462 [==============================] - 18s - loss: 2.1013 - acc: 0.2695 - val_loss: 2.4229 - val_acc: 0.2695
Epoch 8/50
1462/1462 [==============================] - 18s - loss: 1.943

1462/1462 [==============================] - 6s - loss: 0.8338 - acc: 0.6792 - val_loss: 2.0378 - val_acc: 0.4012
Epoch 29/50
1462/1462 [==============================] - 6s - loss: 0.7681 - acc: 0.7093 - val_loss: 2.0108 - val_acc: 0.3653
Epoch 30/50
1462/1462 [==============================] - 6s - loss: 0.7916 - acc: 0.7052 - val_loss: 2.0218 - val_acc: 0.3473
Epoch 31/50
1462/1462 [==============================] - 6s - loss: 0.7809 - acc: 0.6874 - val_loss: 1.9522 - val_acc: 0.4192
Epoch 32/50
1462/1462 [==============================] - 6s - loss: 0.7518 - acc: 0.7161 - val_loss: 1.9700 - val_acc: 0.4072
Epoch 33/50
1462/1462 [==============================] - 6s - loss: 0.7207 - acc: 0.7168 - val_loss: 1.9832 - val_acc: 0.4072
Epoch 34/50
1462/1462 [==============================] - 6s - loss: 0.7022 - acc: 0.7298 - val_loss: 2.0248 - val_acc: 0.3533
Epoch 35/50
1462/1462 [==============================] - 6s - loss: 0.6867 - acc: 0.7141 - val_loss: 2.0566 - val_acc: 0.3533
Epoc

In [14]:
model.load_weights(weight_path)
model.evaluate(test_data, test_labels, batch_size=32, verbose=1, sample_weight=None)

171/171 [==============================] - 0s     


[2.1310109171951028, 0.41520467766544278]